In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import random
from userclass import User
from classifierdto import ClassifierResultDto
from Utils import recepten_bij_tag, tags_bij_recept
from CreateUsers import createSimulatedUsers
from CreateMatrix import createTrainMatrix, createValidateOrTestMatrix
from ClassifierUtils import testClassifiers
import json

In [2]:
ingredientsFile = "ingredients"
ingredientsdf = pd.read_csv(f"/data/foodboost/{ingredientsFile}.csv", index_col=0)

In [3]:
%store -r

In [4]:
diner_recepten = diner_recepten_bonno

In [5]:
def runTestsForUsers(NumberOfUsersPerTag, NumberOfRecipes, tags):
    Results = pd.DataFrame(columns = ['Tag', 'CrossTag', 'BestClassifier', 'Accuracy', 'RecallScore', 'ConfusionMatrix'])
    for tag in tags:
        if (len(recepten_bij_tag(tag)) > NumberOfRecipes):
            filteredTags = list(filter(lambda compareTag: compareTag != tag, tags))
            TrainUsers, ValidateUsers, TestUsers = createSimulatedUsers(diner_recepten, NumberOfUsersPerTag, NumberOfRecipes, [tag])
            for crossTag in filteredTags:
                if (len(recepten_bij_tag(crossTag)) > NumberOfRecipes):
                    
                    print('\n')
                    print("Testing " + tag + " versus " + crossTag)
                    print('\n')

                    TotalTrainMatrix = createTrainMatrix(TrainUsers, [crossTag], diner_recepten, compareToTags = True)
                    #print(TotalTrainMatrix)
                    TotalValidateMatrix = createValidateOrTestMatrix(ValidateUsers, [crossTag], diner_recepten, compareToTags = True)
                    #print(TotalValidateMatrix)
                    
                    
                    y_train = TotalTrainMatrix['y'].to_list()
                    y_validate = TotalValidateMatrix['y'].to_list()
                    
                    TotalTrainMatrix = TotalTrainMatrix.drop(['y', 'Randomgerecht', 'one out'], axis=1)
                    TotalValidateMatrix = TotalValidateMatrix.drop(['y'], axis=1)

                    Classifier = testClassifiers(TotalTrainMatrix, TotalValidateMatrix, y_train, y_validate)
                    ResultDict = {'Tag': tag, 
                              'CrossTag': crossTag, 
                              'BestClassifier': Classifier.Name, 
                              'Accuracy': Classifier.Accuracy,
                              'RecallScore': Classifier.RecallScore}
                              #'ConfusionMatrix': Classifier.ConfusionMatrix.flatten().tolist()}
                    Result = pd.DataFrame(data=ResultDict, columns = ['Tag', 'CrossTag', 'BestClassifier', 'Accuracy', 'RecallScore', 'ConfusionMatrix'])
                    Result.at[0, 'ConfusionMatrix'] = Classifier.ConfusionMatrix
                    display(Result)
                    Results = pd.concat([Results, Result], axis = 0, ignore_index=True)
                else:
                    print("Skipped cross tag " + str(tag) + " because recipes list is smaller than " + str(NumberOfRecipes))
        else:
            print("Skipped tag " + str(tag) + " because recipes list is smaller than " + str(NumberOfRecipes))
    return Results

In [6]:
NumberOfUsersPerTag = 100
NumberOfRecipes = 10

#keukens = ['aziatisch', 'hollands', 'mexicaans', 'italiaans', 'mediterraan']
keukens = ['aziatisch', 'hollands']
Results = runTestsForUsers(NumberOfUsersPerTag, NumberOfRecipes, keukens)



Testing aziatisch versus hollands




,Tag,CrossTag,BestClassifier,Accuracy,RecallScore,ConfusionMatrix
0,aziatisch,hollands,Multinomial Naive Bayes,1.0,1.0,"[[60, 0], [0, 60]]"




Testing hollands versus aziatisch




,Tag,CrossTag,BestClassifier,Accuracy,RecallScore,ConfusionMatrix
0,hollands,aziatisch,Multinomial Naive Bayes,1.0,1.0,"[[60, 0], [0, 60]]"


In [7]:
display(Results)
# Results.to_csv('Data/TagComparisonScoreSpecific.csv')

,Tag,CrossTag,BestClassifier,Accuracy,RecallScore,ConfusionMatrix
0,aziatisch,hollands,Multinomial Naive Bayes,1.0,1.0,"[[60, 0], [0, 60]]"
1,hollands,aziatisch,Multinomial Naive Bayes,1.0,1.0,"[[60, 0], [0, 60]]"
